In [1]:
from pptx import Presentation

In [2]:
import sqlite3

In [3]:
prs = Presentation('NLP-220-Lecture2-2024-Jalal-v.1.pptx')


In [57]:
slide_data = []
for page, slide in enumerate(prs.slides):
   #print(prs.slides.index(slide))
    page_text = []
    for shape in slide.shapes:

        if shape.has_text_frame:
            t = shape.text_frame
            try:
                page_text.append(t.text)
            except Exception as e:
                print(e)
    slide_data.append((page, ' '.join(page_text)))

print(slide_data[3])

(3, 'CSV Parsing  \n \n The separator character is called a delimiter, and the comma is not the only one used. \n\nOther popular delimiters include the tab (\\t), colon (:) and semi-colon (;) characters. \n\nProperly parsing a CSV file requires us to know which delimiter is being used. https://realpython.com/python-csv/')


In [71]:
def get_text_from_pptx(pptx_file:str):
    file_name = pptx_file.split("/")[-1]
    print(file_name)
    prs = Presentation('NLP-220-Lecture2-2024-Jalal-v.1.pptx')

    slide_data = []
    for page, slide in enumerate(prs.slides):
    #print(prs.slides.index(slide))
        page_text = []
        for shape in slide.shapes:

            if shape.has_text_frame:
                t = shape.text_frame
                try:
                    page_text.append(t.text)
                except Exception as e:
                    print(e)
        slide_data.append((file_name, page, ' '.join(page_text)))

    return slide_data

import os
slide_text = []
for file in os.listdir('./'):
    if file.find('.pptx') != -1:
        slide_text = get_text_from_pptx(file)


NLP-220-Lecture2-2024-Jalal-v.1.pptx


In [87]:
# Create a connection to SQLite
conn = sqlite3.connect('searchable.db')

# Enable FTS5 extension (SQLite3 supports it by default in modern versions)
conn.execute('''
CREATE VIRTUAL TABLE IF NOT EXISTS records USING fts5(
            file_name NOT NULL PRIMARY KEY, 
            page NOT NULL UNIQUE, 
            content);
''')

print("FTS5 table created.")

def add_record(conn, data):
    """Insert a record into the FTS5 table."""
    conn.executemany('INSERT INTO records(file_name, page, content) VALUES (?, ?, ?);', data)
    conn.commit()

print(slide_text)


OperationalError: parse error in "file_name NOT NULL PRIMARY KEY"

In [73]:
add_record(conn, slide_text)

In [ ]:
def search_records(conn, query):
    """Search for records matching the query."""
    cursor = conn.execute('SELECT file_name, page, content FROM records WHERE records MATCH ?;', (query,))
    return [row for row in cursor.fetchall()]




('NLP-220-Lecture2-2024-Jalal-v.1.pptx', 44, 'Installing NLTK Download and Install\nhttp://nltk.org/install.html\nDownload NLTK data\n>>> import nltk\n>>> nltk.download()\n\nYou will need to import the necessary modules to create objects and call member functions\nimport ~ include objects from pre-built packages\n\nFreqDist, ConditionalFreqDist are in nltk.probability\n\nPlaintextCorpusReader is in nltk.corpus\n')


In [85]:
OKCYAN = '\033[96m'
WARNING = '\033[93m'
UNDERLINE = '\033[4m'
BOLD = '\033[1m'
print(f'{BOLD}{WARNING} Hello')

 Hello


In [88]:
import sqlite3

# Connect to the database
conn = sqlite3.connect(':memory:')  # Use an in-memory database for testing
cursor = conn.cursor()

# Create a regular table with a primary key
cursor.execute('''
CREATE TABLE items (
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    description TEXT
)
''')

# Create the FTS5 virtual table
cursor.execute('''
CREATE VIRTUAL TABLE items_fts USING fts5(
    title,
    description,
    content='items',  -- Links the FTS table to the regular table
    content_rowid='id'
)
''')

# Insert a row into the regular table and the FTS5 table
cursor.execute('''
INSERT INTO items (id, title, description) VALUES (?, ?, ?)
''', (1, 'Example Title', 'This is an example description.'))

# Update the FTS5 table to reflect the new row
cursor.execute('''
INSERT INTO items_fts (rowid, title, description)
SELECT id, title, description FROM items
''')

# Search using the FTS5 table
cursor.execute('''
SELECT items.id, items.title, items.description
FROM items
JOIN items_fts ON items.id = items_fts.rowid
WHERE items_fts MATCH 'example'
''')

# Fetch and print results
results = cursor.fetchall()
print("Search results:", results)

# Close the connection
conn.close()


Search results: [(1, 'Example Title', 'This is an example description.')]
